In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LinearRegression
from scipy import stats

In [2]:
# load data
sales = pd.read_csv("../../ML Data & Script/kc_house_data.csv")
sales.head()

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,...,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
0,7129300520,20141013T000000,221900.0,3,1.00,1180,5650,1.0,0,0,...,7,1180,0,1955,0,98178,47.5112,-122.257,1340,5650
1,6414100192,20141209T000000,538000.0,3,2.25,2570,7242,2.0,0,0,...,7,2170,400,1951,1991,98125,47.7210,-122.319,1690,7639
2,5631500400,20150225T000000,180000.0,2,1.00,770,10000,1.0,0,0,...,6,770,0,1933,0,98028,47.7379,-122.233,2720,8062
3,2487200875,20141209T000000,604000.0,4,3.00,1960,5000,1.0,0,0,...,7,1050,910,1965,0,98136,47.5208,-122.393,1360,5000
4,1954400510,20150218T000000,510000.0,3,2.00,1680,8080,1.0,0,0,...,8,1680,0,1987,0,98074,47.6168,-122.045,1800,7503


In [3]:
def get_numpy_data(data, features, output):
    # add a column with all ones to a dataframe
    data['constant'] = 1 
    # add the column 'constant' to the front of the features list so that we can extract it along with the others:
    # this is how you combine two lists
    features = ['constant'] + features 
    # get a dataframe with the selected features and convert to numpy matrix
    X = data[features].values
    # get output column(pandas series)
    output = data[output]
    # convert pandas series to numpy array
    y = output.values
    return(X, y)

In [4]:
# Checking the data at the first row for sqft
print(sales['sqft_living'][0])
#Checking the output of the first row
print(sales['price'][0])

1180
221900.0


In [5]:
# the [] around 'sqft_living' makes it a list
(X, y) = get_numpy_data(sales, ['sqft_living'], 'price')
# this accesses the first row of the data the ':' indicates 'all columns'
print("first row, features",  X[0, :].reshape(1,2))
# and the corresponding output
print("first row, output", y[0].reshape(1,1)) 
print(X[0,:].reshape(1,2))

first row, features [[   1 1180]]
first row, output [[221900.]]
[[   1 1180]]


In [6]:
# shape of X and y
print(X.shape)
y = y.reshape(-1,1)
print(y.shape)

(21613, 2)
(21613, 1)


In [7]:
# predict output given weights
# the example weights
my_weights = np.array([1., 1.]) 
# we'll use the first data point
first_data_point = X[0,]
# 1 * 1 + 1 * 1180
y_pred = np.dot(first_data_point, my_weights)
print(y_pred)

1181.0


In [8]:
# calculate prediction for the whole data
def predict_output(data_matrix, weights):
    # assume feature_matrix is a numpy matrix containing the features as columns and weights is a corresponding numpy array
    # create the predictions vector by using np.dot()
    pred = np.dot(data_matrix, weights)
    return(pred)

In [9]:
# test prediction
# (21613, 2).(2,1)
test_pred = predict_output(X, my_weights)
# should be 1181.0
print(test_pred[0])
# should be 2571.0
print(test_pred[1]) 

1181.0
2571.0


In [10]:
# compute derivative
def feature_derivative(errors, feature):
    # Assume that errors and feature are both numpy arrays of the same length (number of data points)
    # compute twice the dot product of these vectors as 'derivative' and return the value
    derivative=2* (np.dot(errors,feature))
    return(derivative)

In [11]:
# test feature derivative
(X, y) = get_numpy_data(sales, ['sqft_living'], 'price') 
my_weights = np.array([0., 0.]) # this makes all the predictions 0
yhat = predict_output(X, my_weights) 
# just like SFrames 2 numpy arrays can be elementwise subtracted with '-': 
# prediction errors in this case is just the -example_output
errors = yhat - y 
# let's compute the derivative with respect to 'constant', the ":" indicates "all rows"
feature = X[:,0] 
derivative = feature_derivative(errors, feature)
print(derivative)
# should be the same as derivative
print(-np.sum(y) * 2) 

-23345850016.0
-23345850016.0


In [12]:
from math import sqrt
def regression_gradient_descent(X, y, initial_weights, step_size, tolerance):
    converged = False 
    # make sure it's a numpy array
    weights = np.array(initial_weights) 
    while not converged:
        # compute the predictions based on feature_matrix and weights using your predict_output() function
        yhat = predict_output(X, weights)
        # compute the errors as predictions - output
        errors = yhat - y
        # initialize the gradient sum of squares
        gradient_sum_squares = 0 
        # while we haven't reached the tolerance yet, update each feature's weight
        # loop over each weight
        for i in range(len(weights)): 
            # Recall that feature_matrix[:, i] is the feature column associated with weights[i]
            # compute the derivative for weight[i]:
            derivative = feature_derivative(errors, X[:, i])
            # add the squared value of the derivative to the gradient sum of squares (for assessing convergence)
            gradient_sum_squares += derivative * derivative
            # subtract the step size times the derivative from the current weight
            weights[i] -= step_size * derivative
        # compute the square-root of the gradient sum of squares to get the gradient magnitude:
        gradient_magnitude = sqrt(gradient_sum_squares)
        if gradient_magnitude < tolerance:
            converged = True
    return(weights)

In [13]:
# run gradient descent, simple linear regression
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(sales, sales['price'], test_size=0.2, random_state=0)
# let's test out the gradient descent
(X, y) = get_numpy_data(X_train, ['sqft_living'], 'price')
initial_weights = np.array([-47000., 1.])
step_size = 7e-12
tolerance = 2.5e7
simple_weights = regression_gradient_descent(X, y, initial_weights, step_size, tolerance)
print(simple_weights)

[-46999.88720259    283.46383063]


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


Very close to the weights obtained using sklearn
* Intercept: -48257.063591028564
* Slope: 283.96855715512993

In [14]:
# test data, simple features
(X_test, y_test) = get_numpy_data(X_test, ['sqft_living'], 'price')
y_hat_model_one = predict_output(X_test, simple_weights)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [15]:
# predicited house price for the first house
y_hat_model_one[0]

358353.39059137006

In [16]:
# running multiple regression
# sqft_living15 is the average squarefeet for the nearest 15 neighbors.
X_train, X_test, y_train, y_test = train_test_split(sales, sales['price'], test_size=0.2, random_state=0)
adv_features = ['sqft_living', 'sqft_living15']  
(X_train, y_train) = get_numpy_data(X_train, adv_features, 'price')
initial_weights = np.array([-100000., 1., 1.])
step_size = 4e-12
tolerance = 1e9
adv_weights = regression_gradient_descent(X_train, y_train, initial_weights, step_size, tolerance)
print(adv_weights)


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


[-9.99999757e+04  2.47055837e+02  6.47974873e+01]


In [17]:
# test on a test set
(X_test, y_test) = get_numpy_data(X_test, adv_features, 'price')
y_hat_model_two = predict_output(X_test, adv_weights)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [18]:
# model2 prediction for the first house
y_hat_model_two[0]

345950.2780906761

In [19]:
y_test[0]

297000.0

In [20]:
# difference with model one
print(y_test[0] - y_hat_model_one[0])
# difference with model two
print(y_test[0] - y_hat_model_two[0])

-61353.39059137006
-48950.27809067612


In [21]:
# RSS
# model_one RSS
print(((y_hat_model_one - y_test) ** 2).sum())
# model_two RSS
print(((y_hat_model_two - y_test) ** 2).sum())

267729995270518.6
262684098596668.75


model_one performed better for the first house on the testset, but overall model_two has lower RSS

### F-test
* unexplained model variance:

$$SSE_F=\sum(y_i-\hat{y}_i)^2$$

* unexplained variance in reduced model:

$$SSE_R=Var_y = \sum(y_i-\bar{y})^2$$

 * number of parameters in the model:

$$p_F = 2 (\alpha \text{ and } \beta)$$

 * number of parameters in the reduced model:

$$p_R = 1 (\alpha)$$

 * number of datapoints:

$$n$$

 * degrees of freedom of $SSE_F$:

$$df_F = n - p_F$$

 * degrees of freedom of $SSE_R$:

$$df_R = n - p_R$$

These pieces come together to give us the full equation for the F-test:

$$F=\dfrac{SSE_F-SSE_R}{df_F-df_R}÷\dfrac{SSE_F}{df_F}$$

Read these: 

http://facweb.cs.depaul.edu/sjost/csc423/documents/f-test-reg.htm

http://eric.univ-lyon2.fr/~ricco/tanagra/fichiers/en_Tanagra_Calcul_P_Value.pdf

https://www.youtube.com/watch?v=JBXkKOPZDsc&ab_channel=mathtutordvd

In [41]:
# calcuate FValue from scratch
((y_hat_model_one - y_test) ** 2).sum()

SSEf = ((y_hat_model_one - y_test) ** 2).sum()
SSEr = ((y_test.mean() - y_test) ** 2).sum()
n = y_test.shape[0]
pF = 2
pR = 1
dfF = n - pF
dfR = n - pR
print("degree of freedom 1: ", dfR)
print("degree of freedom 2: ", dfF)

F = ((SSEf - SSEr) / (dfF - dfR)) * (dfF / SSEf)
print("fstatistic: ", F)
# the null hypothesis: all of the regression parameters are zero
import scipy
#Or whatever you want your alpha to be.
alpha = 0.05 
pvalue = 1 - scipy.stats.f.cdf(F, dfR, dfF)
print(pvalue, pvalue > alpha)
# Reject the null hypothesis 

degree of freedom 1:  4322
degree of freedom 2:  4321
fstatistic:  3976.417886342636
1.1102230246251565e-16 False


In [42]:
# calcuate FValue from scratch
((y_hat_model_one - y_test) ** 2).sum()

SSEf = ((y_hat_model_two - y_test) ** 2).sum()
SSEr = ((y_test.mean() - y_test) ** 2).sum()
n = y_test.shape[0]
pF = 3
pR = 1
dfF = n - pF
dfR = n - pR
print("degree of freedom 1: ", dfR)
print("degree of freedom 2: ", dfF)

F = ((SSEf - SSEr) / (dfF - dfR)) * (dfF / SSEf)
print("fstatistic: ", F)
# the null hypothesis: all of the regression parameters are zero
import scipy
#Or whatever you want your alpha to be.
alpha = 0.05 
pvalue = 1 - scipy.stats.f.cdf(F, dfR, dfF)
print(pvalue, pvalue > alpha)
# Reject the null hypothesis 

degree of freedom 1:  4322
degree of freedom 2:  4320
fstatistic:  2067.422888779355
1.1102230246251565e-16 False


In [57]:
# Look the cumulative area up to the statistic is almost one, so remaining area is very close to 0
scipy.stats.f.cdf(F, dfR, dfF )
# 1 - cdf gives the remaining area.
# 

0.9999999999999999

<bound method rv_continuous.cdf of <scipy.stats._continuous_distns.f_gen object at 0x000002424C64A9E8>>